# Assignment 5 - Mountain Car
## Answers

Space X and V(elocity) discretized into 10 BINS each. <br>
Goal reached 64 times for epsilon 0.3 average epoch is 28211.04 <br>
Goal reached 100 times for epsilon 0.2 average epoch is 15536.05 <br>
Goal reached 100 times for epsilon 0.1 average epoch is 5402.58<br>
Goal reached 0 times for epsilon 0.0 <br>

In [1]:
import math
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import pandas as pd

In [2]:
class MountainCarEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 30
    }

    def __init__(self):
        self.min_position = -1.2
        self.max_position = 0.6
        self.max_speed = 0.07
        self.goal_position = 0.5

        self.low = np.array([self.min_position, -self.max_speed])
        self.high = np.array([self.max_position, self.max_speed])

        self.viewer = None

        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(self.low, self.high)

        self.seed()
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid" % (action, type(action))

        position, velocity = self.state
        velocity += (action-1)*0.001 + math.cos(3*position)*(-0.0025)
        velocity = np.clip(velocity, -self.max_speed, self.max_speed)
        position += velocity
        position = np.clip(position, self.min_position, self.max_position)
        if (position==self.min_position and velocity<0): velocity = 0

        done = bool(position >= self.goal_position)
        reward = -1.0        
        if done:
            reward = 10.0

        self.state = (position, velocity)
        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = np.array([self.np_random.uniform(low=-0.6, high=-0.4), 0])
        return np.array(self.state)

    def _height(self, xs):
        return np.sin(3 * xs)*.45+.55
    
    def close(self):
        if self.viewer: self.viewer.close()

In [3]:
class QLearn(object): 
    def __init__(self, Q, policy, legal_actions, actions, gamma, lr): 
        self.Q = Q #Q matrix 
        self.policy = policy 
        self.legal_actions = legal_actions
        self.actions = actions
        self.gamma = gamma
        self.lr = lr

    def q_value(self,s,a): 
        if (s,a) in self.Q: 
            self.Q[(s,a)] 
        else: 
            self.Q[s,a] = 0 
        return self.Q[s,a] 
    
    def action(self, s): 
        if s in self.policy: 
            return self.policy[s] 
        else:
            self.policy[s] = np.random.randint(0,self.legal_actions) 
        return self.policy[s] 
    
    def learn(self,s,a,s1,r,done): 
        if done == False: 
            self.Q[(s,a)] = self.q_value(s,a) + self.lr*(r+self.gamma*max([self.q_value(s1,a1) for a1 in self.actions]) - self.q_value(s,a)) 
        else: 
            self.Q[(s,a)] = self.q_value(s,a) + self.lr*(r - self.q_value(s,a))
            self.q_values = [self.q_value(s,a1) for a1 in self.actions] 
            self.policy[s] = self.actions[self.q_values.index(max(self.q_values))]


In [4]:
def discretize(obs):
       return tuple([int(np.digitize(obs[i], BINS[i])) for i in range(len(N_BINS))])

In [5]:
env = gym.make("MountainCarAssignmentVersion-v0")
#env = gym.make("MountainCar-v0")
s = env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [6]:
legal_actions = env.action_space.n
actions = [0,1,2]
gamma = 0.99
lr = 0.1
num_episodes = 100
epsilon = 0.3
#epsilons = [0.0, 0.1, 0.2, 0.3]
epsilon_decay = 1#0.99
epochs = 100000

In [7]:
N_BINS = [10,10] #10 possible x-positions, and 10 possible velocities
MAX_VALUES = [0.6,0.07]
MIN_VALUES = [-1.2,-.07]
BINS = [np.linspace(MIN_VALUES[i], MAX_VALUES[i], N_BINS[i]) for i in range(len(N_BINS))]


In [8]:
rList = []

Q = {}
policy = {}
QL = QLearn(Q,policy,legal_actions,actions,gamma,lr)

In [9]:
countGoals = 0
episodesEpochsGoal = []

for i in range(num_episodes):
    s_raw = env.reset()
    s = discretize(s_raw)
    rAll = 0
    d = False
    j = 0
    for j in range(epochs):
        #epsilon greedy. to choose random actions initially when Q is all zeros
        if np.random.random() < epsilon:
            a = np.random.randint(0, legal_actions)
            epsilon = epsilon * epsilon_decay
        else:
            a = QL.action(s)
        s1_raw,r,d,_ = env.step(a)
        #if d: r = 10.0
        rAll = rAll + r
        s1 = discretize(s1_raw)
        if d:
            if rAll < -epochs+1:
                r = -epochs
                QL.learn(s,a,s1,r,d)
                print("Failed! Reward %d"%rAll)
            elif rAll > -epochs+1:
                countGoals += 1
                episodesEpochsGoal.append((i+1, j+1))
                print("Passed! Reward %d, episode %d epoch #%d"%(rAll,i+1,j+1))
            break
        QL.learn(s,a,s1,r,d)
        if j == epochs - 1:
            print("Reward %d after full episode#%d"%(rAll,i+1))
        s = s1
env.close()
sumEpochs = 0
for epiEpo in episodesEpochsGoal:
    sumEpochs += epiEpo[1]
avgEpoch = sumEpochs/num_episodes

print("Goal reached {} times for epsilon {} average epoch is {}, at the following (episode, epoch) {}".format(countGoals, epsilon, avgEpoch, episodesEpochsGoal))
#Goal reached 28 times, at the following (episode, epoch) [(4, 43295), (9, 75929), (13, 45001), (15,

Reward -100000 after full episode#1
Reward -100000 after full episode#2
Reward -100000 after full episode#3
Reward -100000 after full episode#4
Reward -100000 after full episode#5
Reward -100000 after full episode#6
Reward -100000 after full episode#7
Reward -100000 after full episode#8
Reward -100000 after full episode#9
Reward -100000 after full episode#10


KeyboardInterrupt: 